# Shallow Neural Network: NumPy vs TensorFlow

This notebook compares a **from-scratch NumPy implementation** and a **TensorFlow implementation** of a shallow neural network with backpropagation.

## NumPy Implementation 

In [1]:

import numpy as np

X = np.array([[1,2],[2,3],[3,4],[4,5],[5,6],[6,7]], dtype=float)
y = np.array([[0],[0],[0],[1],[1],[1]], dtype=float)

m, n = X.shape
np.random.seed(42)

W1 = np.random.randn(n, 4) * 0.01
b1 = np.zeros((1, 4))
W2 = np.random.randn(4, 1) * 0.01
b2 = np.zeros((1, 1))

def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return (Z > 0).astype(float)

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

lr = 0.01
epochs = 1000

for epoch in range(epochs):
    Z1 = X @ W1 + b1
    A1 = relu(Z1)
    Z2 = A1 @ W2 + b2
    A2 = sigmoid(Z2)

    loss = -np.mean(y*np.log(A2+1e-8)+(1-y)*np.log(1-A2+1e-8))

    dZ2 = A2 - y
    dW2 = A1.T @ dZ2 / m
    db2 = np.mean(dZ2, axis=0, keepdims=True)

    dA1 = dZ2 @ W2.T
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = X.T @ dZ1 / m
    db1 = np.mean(dZ1, axis=0, keepdims=True)

    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2

    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


Epoch 0, Loss: 0.6933
Epoch 200, Loss: 0.6887
Epoch 400, Loss: 0.6240
Epoch 600, Loss: 0.5681
Epoch 800, Loss: 0.5146


## TensorFlow Implementation 

In [2]:

import tensorflow as tf

X_tf = tf.constant(X, dtype=tf.float32)
y_tf = tf.constant(y, dtype=tf.float32)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

for epoch in range(1000):
    with tf.GradientTape() as tape:
        y_pred = model(X_tf, training=True)
        loss = loss_fn(y_tf, y_pred)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}")


C:\Users\user\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 0, Loss: 0.9658
Epoch 200, Loss: 0.3170
Epoch 400, Loss: 0.1258
Epoch 600, Loss: 0.0662
Epoch 800, Loss: 0.0365
